# Práctica 1: LiDAR Clustering



## Índice

1. Integrantes del equipo

2. Librerías

3. Carga de Datos <br>

    3.1. Dataset carretera vacía <br>

    3.2.  Filtrado de datos <br>
    
    3.3.  Dataset Coche/Moto <br>

5. Procesamiento de datos
6. Clustering DBSCAN
7. Clustering KMeans
8. Conclusiones


## Integrantes del equipo

* Alejandro Cortijo Benito
* Alejandro García Mota

Este notebook tiene como objetivo clusterizar datos LiDAR para detectar el número de vehículos que hay en cada muestra, para resolverlo usaremos los dos algoritmos de clustering vistos en la asignatura: **DBSCAN** y **KMeans**. 

Cabe destacar que en este cuaderno, presentaremos los resultados usando una de las muestras más complejas, la muestra de dos coches y una moto.

## <a id='imports'></a>Librerias

In [37]:
# Procesamiento de datos
import numpy as np
import pandas as pd

# Procesamiento de datos
from sklearn.linear_model import RANSACRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Clustering
from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import pairwise_distances_argmin_min

# Visualización de datos
import plotly.express as px
import plotly.graph_objects as go

## <a id='dataset'></a>Carga de datos

### Dataset carretera vacía

In [38]:
df = pd.read_csv('./data/carretera.csv')
df.tail(10)

,x,y,z,intensity,t,reflectivity,ring,ambient,range
131062,-0.0,-0.0,-0.0,0.0,586440720,0,127,0,0
131063,-0.0,-0.0,-0.0,0.0,586440720,0,127,0,0
131064,-0.0,-0.0,-0.0,0.0,586440720,0,127,0,0
131065,-0.0,-0.0,-0.0,0.0,586440720,0,127,0,0
131066,-0.0,-0.0,-0.0,0.0,586440720,0,127,0,0
131067,-0.0,-0.0,-0.0,0.0,586440720,0,127,0,0
131068,-0.0,-0.0,-0.0,0.0,586440720,0,127,0,0
131069,-0.0,-0.0,-0.0,0.0,586440720,0,127,0,0
131070,-0.0,-0.0,-0.0,0.0,586440720,0,127,0,0
131071,-0.0,-0.0,-0.0,0.0,586440720,0,127,0,0


In [ ]:
fig = px.scatter(df, x='x', y='z', color='ring')
fig.show()

<img src="./outputs_clustering/0.png"/>

Una vez almacenamos la información de la carretera en un DataFrame y comprendemos la estructura de los datos a los que nos estamos enfrentando, procedemos a seleccionar una **región de interés (ROI)** adecuada para nuestro caso de estudio. Para optimizar la detección de vehículos, limpiamos los datos de la carretera limitando el campo visual a una zona de 4 metros de ancho y 30 metros de largo. 

Esta selección se realizó de forma empírica, tras probar diferentes configuraciones y determinar que esta distancia proporcionaba la mejor relación entre la precisión en la detección y la relevancia de los datos para nuestro objetivo.

Además de establecer un ROI, hemos eliminado aquellos puntos de `y` mayores a 2, con el fin de suprimir el ruido generado por la posición elevada del sensor LiDAR sobre el pórtico.

### Preprocesamiento de los datos

In [40]:
def filter_points(df_input):
    widht = 4
    lenght = 30

    df_filtered = df_input[~((df_input['x'] == 0) & (df_input['y'] == 0) & (df_input['z'] == 0))]
    df_filtered = df_filtered[(df_filtered['z'] > -widht) & (df_filtered['z'] < widht) & (df_filtered['x'] > -lenght) & (df_filtered['x'] < lenght)]
    df_filtered = df_filtered[df_filtered['y'] > 2]
    return df_filtered

In [41]:
print(f"Number of rows before removing: {len(df)}")

df_cleaned = filter_points(df)
print(f"Number of rows after removing: {len(df_cleaned)}")

Number of rows before removing: 131072
Number of rows after removing: 32470


In [ ]:
fig = px.scatter(df_cleaned, x='x', y='z', color='ring')
fig.show()

<img src="./outputs_clustering/1.png"/>

Una vez tenemos cargado y procesado nuestro `ground truth` pasamos a ver la muestra principal donde vamos a realizar el clustering.

### Dataset Coche/Moto

De manera similar al conjunto de datos anterior, hacemos una limpieza inicial de los datos.

In [ ]:
df_car = pd.read_csv('./data/coche_coche_moto.csv')
df_car_filtered = filter_points(df_car)

fig = px.scatter(df_car_filtered, x='x', y='z', color='ring')
fig.show()

<img src="./outputs_clustering/2.png"/>

## Procesamiento de los datos

Como podemos observar todavía tenemos muchos puntos, demasiados si quisieramos clusterizar. Para abordar este problema nos planteamos varias estrategias. Inicialmente, consideramos eliminar aquellos puntos que fueran prácticamente iguales al `ground truth`, utilizando una tolerancia determinada, otra idea fue afrontar el problema como una regresión.

Finalmente, combinamos lo mejor de ambas aproximaciones. Primero, empleamos el algoritmo **RANSAC (RANdom SAmple Consensus)** para ajustar un plano a los datos correspondientes a la carretera, ya que esta superficie representa la mayor parte de los puntos en la muestra. Una vez ajustado este plano, filtramos los puntos pertenecientes a nuestra muestra objetivo (2 coches y 1 moto), utilizando una tolerancia de 0.1 para determinar qué puntos no pertenecen al plano ajustado y así descartarlos.

In [44]:
# Extrayendo las columnas relevantes (coordenadas 'x' y 'z') como variables independientes
X = df_cleaned[['x', 'z']]
y = df_cleaned['y']

ransac = make_pipeline(PolynomialFeatures(degree=1), RANSACRegressor())

# Ajustando el modelo a los datos (entrenando el modelo)
ransac.fit(X, y)

# Prediciendo los valores de 'y' (altura) a partir de las coordenadas 'x' y 'z' usando el modelo ajustado
y_pred = ransac.predict(X)

In [ ]:
threshold = 0.1  
df_car_filtered['y_pred'] = ransac.predict(df_car_filtered[['x', 'z']])
df_car_filtered_plane = df_car_filtered[np.abs(df_car_filtered['y'] - df_car_filtered['y_pred']) > threshold]


fig = px.scatter(df_car_filtered_plane, x='x', y='z', color='ring')
fig.show()


<img src="./outputs_clustering/3.png"/>

Ahora ya somos capaces de distinguir visualmente los vehículos, además de algún outlayer. Haciendo uso de este DataFrame filtrado empecemos a clusterizar.

## Clustering con DBSCAN

In [ ]:
# Extrayendo las columnas relevantes ('x', 'y', 'z') para realizar el clustering
X = df_car_filtered_plane[['x', 'y', 'z']]
df_clustered_dbscan = X.copy()

# Configuramos DBSCAN
dbscan = DBSCAN(eps=2, min_samples=5)

# Aplicamos DBSCAN
y_dbscan = dbscan.fit_predict(X)
df_clustered_dbscan['cluster'] = y_dbscan

# Eliminamos los puntos clasificados como ruido (donde el clúster es -1)
df_clustered_dbscan = df_clustered_dbscan[df_clustered_dbscan['cluster'] != -1]

# Visualizando los datos agrupados en un gráfico de dispersión, coloreados por clústeres
fig = px.scatter(df_clustered_dbscan, x='x', y='z', color='cluster')
fig.show()

<img src="./outputs_clustering/4.png"/>

In [ ]:
# Visualización 3D
df_result_dbscan = pd.DataFrame()
df_result_dbscan['x'] = df_clustered_dbscan['z']
df_result_dbscan['y'] = -df_clustered_dbscan['x']
df_result_dbscan['z'] = -df_clustered_dbscan['y']
df_result_dbscan['cluster'] = df_clustered_dbscan['cluster']

fig = go.Figure(data=[go.Scatter3d(
    x=df_result_dbscan['x'],
    y=df_result_dbscan['y'],
    z=df_result_dbscan['z'],
    mode='markers',
    marker=dict(
        size=1,
        color=df_clustered_dbscan['cluster'],
        colorscale='Viridis',
        opacity=0.8
    )
)])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data',
        camera=dict(
                eye=dict(x=2, y=2, z=2)
            )
    ),
    title='DBSACAN Scatter Plot of Clustered Data'
)

fig.show()

<img src="./outputs_clustering/5.png"/>

In [ ]:
# Calcualr bounding boxes para cada cluster
bounding_boxes = {}

for cluster in np.unique(y_dbscan)[1:]:
    cluster_points = df_result_dbscan[df_result_dbscan['cluster'] == cluster]
    min_x, max_x = cluster_points['x'].min(), cluster_points['x'].max()
    min_y, max_y = cluster_points['y'].min(), cluster_points['y'].max()
    min_z, max_z = cluster_points['z'].min(), cluster_points['z'].max()
    bounding_boxes[cluster] = {
        'min_x': min_x, 'max_x': max_x,
        'min_y': min_y, 'max_y': max_y,
        'min_z': min_z, 'max_z': max_z
    }

fig_dbscan = go.Figure()

# Añadir puntos originales
fig_dbscan.add_trace(go.Scatter3d(
    x=df_result_dbscan['x'],
    y=df_result_dbscan['y'],
    z=df_result_dbscan['z'],
    mode='markers',
    marker=dict(
        size=1,
        color=df_result_dbscan['cluster'],
        colorscale='Viridis',
        opacity=0.8
    )
))

for cluster, bbox in bounding_boxes.items():
    fig_dbscan.add_trace(go.Scatter3d(
        x=[bbox['min_x'], bbox['max_x'], bbox['max_x'], bbox['min_x'], bbox['min_x'], bbox['min_x'], bbox['max_x'], bbox['max_x'], bbox['min_x'], bbox['min_x'], bbox['max_x'], bbox['max_x'], bbox['max_x'], bbox['max_x'], bbox['min_x'], bbox['min_x'], bbox['min_x']],
        y=[bbox['min_y'], bbox['min_y'], bbox['max_y'], bbox['max_y'], bbox['min_y'], bbox['min_y'], bbox['min_y'], bbox['max_y'], bbox['max_y'], bbox['max_y'], bbox['max_y'], bbox['max_y'], bbox['min_y'], bbox['min_y'], bbox['min_y'], bbox['min_y'], bbox['max_y']],
        z=[bbox['min_z'], bbox['min_z'], bbox['min_z'], bbox['min_z'], bbox['min_z'], bbox['max_z'], bbox['max_z'], bbox['max_z'], bbox['max_z'], bbox['min_z'], bbox['min_z'], bbox['max_z'], bbox['max_z'], bbox['min_z'], bbox['min_z'], bbox['max_z'], bbox['max_z']],
        mode='lines',
        line=dict(color='red', width=2),
        name=f'Vehículo {cluster}'
    ))

fig_dbscan.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data',
        camera=dict(
                eye=dict(x=2, y=2, z=2)
            )
    ),
    title='DBSCAN Scatter Plot with Bounding Boxes'
)

fig_dbscan.show()

<img src="./outputs_clustering/6.png"/>

## Clustering con KMeans

De manera analoga al DBSCAN procedemos a clusterizar usando KMeans.

In [ ]:
X = df_car_filtered_plane[['x', 'y', 'z']]

kmeans = KMeans(n_clusters=3, random_state=0)
y_kmeans = kmeans.fit_predict(X)

centroids = kmeans.cluster_centers_

# Eliminar ruido que el KMeans ha interpretado como cluster
_, distances = pairwise_distances_argmin_min(X, centroids)

# Max distance
distancia_max = 4

mask = distances <= distancia_max
df_clustered_kmeans = X[mask].copy()
df_clustered_kmeans['cluster'] = y_kmeans[mask]

fig = px.scatter(df_clustered_kmeans, x='x', y='z', color='cluster')
fig.show()

<img src="./outputs_clustering/7.png"/>

In [ ]:
df_result_kmeans = pd.DataFrame()
df_result_kmeans['x'] = df_clustered_kmeans['z']
df_result_kmeans['y'] = -df_clustered_kmeans['x']
df_result_kmeans['z'] = -df_clustered_kmeans['y']
df_result_kmeans['cluster'] = df_clustered_kmeans['cluster']

fig = go.Figure(data=[go.Scatter3d(
    x=df_result_kmeans['x'],
    y=df_result_kmeans['y'],
    z=df_result_kmeans['z'],
    mode='markers',
    marker=dict(
        size=1,
        color=df_clustered_kmeans['cluster'],
        colorscale='Viridis',
        opacity=0.8
    )
)])

fig.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data',
        camera=dict(
                eye=dict(x=2, y=2, z=2)  # Adjust camera angle
            )
    ),
    title='KMeans Scatter Plot of Clustered Data'
)

fig.show()

<img src="./outputs_clustering/8.png"/>

In [ ]:
# Calculate bounding boxes for each cluster
bounding_boxes = {}

for cluster in np.unique(y_kmeans):
    
    cluster_points = df_result_kmeans[df_result_kmeans['cluster'] == cluster]
    min_x, max_x = cluster_points['x'].min(), cluster_points['x'].max()
    min_y, max_y = cluster_points['y'].min(), cluster_points['y'].max()
    min_z, max_z = cluster_points['z'].min(), cluster_points['z'].max()
    bounding_boxes[cluster] = {
        'min_x': min_x, 'max_x': max_x,
        'min_y': min_y, 'max_y': max_y,
        'min_z': min_z, 'max_z': max_z
    }

fig_kmeans = go.Figure()

# Add original data points
fig_kmeans.add_trace(go.Scatter3d(
    x=df_result_kmeans['x'],
    y=df_result_kmeans['y'],
    z=df_result_kmeans['z'],
    mode='markers',
    marker=dict(
        size=1,
        color=df_result_kmeans['cluster'],
        colorscale='Viridis',
        opacity=0.8
    )
))

for cluster, bbox in bounding_boxes.items():
    fig_kmeans.add_trace(go.Scatter3d(
        x=[bbox['min_x'], bbox['max_x'], bbox['max_x'], bbox['min_x'], bbox['min_x'], bbox['min_x'], bbox['max_x'], bbox['max_x'], bbox['min_x'], bbox['min_x'], bbox['max_x'], bbox['max_x'], bbox['max_x'], bbox['max_x'], bbox['min_x'], bbox['min_x'], bbox['min_x']],
        y=[bbox['min_y'], bbox['min_y'], bbox['max_y'], bbox['max_y'], bbox['min_y'], bbox['min_y'], bbox['min_y'], bbox['max_y'], bbox['max_y'], bbox['max_y'], bbox['max_y'], bbox['max_y'], bbox['min_y'], bbox['min_y'], bbox['min_y'], bbox['min_y'], bbox['max_y']],
        z=[bbox['min_z'], bbox['min_z'], bbox['min_z'], bbox['min_z'], bbox['min_z'], bbox['max_z'], bbox['max_z'], bbox['max_z'], bbox['max_z'], bbox['min_z'], bbox['min_z'], bbox['max_z'], bbox['max_z'], bbox['min_z'], bbox['min_z'], bbox['max_z'], bbox['max_z']],
        mode='lines',
        line=dict(color='red', width=2),
        name=f'Vehículo {cluster}'
    ))

fig_kmeans.update_layout(
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data',
        camera=dict(
                eye=dict(x=2, y=2, z=2)  # Adjust camera angle
            )
    ),
    title='KMeans Scatter Plot with Bounding Boxes'
)

fig_kmeans.show()

<img src="./outputs_clustering/9.png"/>

## Conclusiones

En esta práctica, se han aplicado los algoritmos K-means y DBSCAN para clusterizar vehículos de muestras recogidas con LiDAR. Ambos métodos tienen fortaleza pero una ventaja clave del DBSCAN frente al K-means es que no requiere predefinir el número de clústeres. Esto nos puede resultar bastante útil cuando no teniamos conocimientos previos de la escena que analizabamos (como en la segunda parte de la práctica haciendo el tracking). También destaca por su capacidad de detectar y manejar ruido de manera robusta (cluster -1). 

En resumen, DBSCAN parece ser la solución más completa y eficaz para este tipo de aplicaciones, donde es común enfrentar datos ruidosos y no se tiene información previa sobre la cantidad o forma de los clústeres.